# Spaceship around a black hole - a theoretical approach

Before creating the game, I'll introduce and compute some quantities in order to study the system behavior. We will by describing the simplified model we are using.

We have a spaceship of mass $m$ revolving around a black hole of mass $M$. The black hole has a Schwarzschild radius $R_S$, relating to the event horizon, defined as 

$$ [1] \rightarrow   \;\;\;      R_S = \frac{2GM}{c^2}$$

Where $G \approx 6.67\cdot10^{-11}$ is the universal gravitional constant and $c$ the speed of light. 

## On the event horizon

Recall the expression of [1]. One can define in orbital mechanics the _escape velocity_ $v_e$ to be the velocity at which an object should move in order to leave the gravitional attraction due to a body. Mathematically, 

$$ v_e = \sqrt{\frac{2GM}{r}} $$

We can easily see that given $r = R_S$, $v_e$ equals to $c$. As $v_e$ is also proportional to $1/r$, a lesser distance from the black hole would mean a higher escape velocity than $c$ needed to leave gravitational effects : it is forbidden by special relativity. 

## Reduced units

To simplify equations, we will work with reduced units by setting $R_S = c = 1$. It follows from [1] that $M = 1/2G$. 

All quantities will be expressed as multiples of $R_S$, $c$ and $M$, with proportional constants noted with greek letters. As such, we define the mass of the spaceship $m = \alpha M$ to be a small fraction of the mass of the black hole. 

## Dynamics

We will use the law of Newton to model the system.

$$ \partial_t \; \vec{p} = \sum \vec{F} $$

Accounting two forces in the right-hand side. The first one is the classical gravitational force 

$$ \vec{F}_G = - G \frac{Mm}{r^2} \vec{e}_r$$

And the second one brings a first order relativistic correction to dynamics,

$$ \vec{F}_R = -\frac{3}{2} m \dot\theta^2 R_S  \vec{e}_r$$

This reduces to a single differential equation in polar coordinates :

$$[2] \rightarrow   \;\;\;      \ddot r = - G \frac{M}{r^2} + (r - \frac{3}{2}R_S \dot\theta ^2) $$

We can approximate a solution by the following expression for $r$,
$$  [3] \rightarrow   \;\;\;      r_\pm (\theta)= \frac{\frac{q}{2} \pm \sqrt{\frac{q^2}{4} - \frac{3}{2}R_S q (1+e\cos\theta) }}{1+e\cos\theta} $$

Where $q$ is the _semi latus rectum_ of the ellipse and $e$ the eccentricity. Given the space region we are, we can expose two solutions. If $r > 3R_S$, we'll use $r_+$ ; $r_-$ otherwise. This approximation will help us studying the orbit paths of the spaceship before relying on numerical computations to solve the system.

## Input parameters

<a href="https://phys.libretexts.org/Bookshelves/Astronomy__Cosmology/Celestial_Mechanics_(Tatum)/09%3A_The_Two_Body_Problem_in_Two_Dimensions/9.04%3A_Kepler's_First_and_Third_Laws_from_Newton's_Law_of_Gravitation"> Resources </a>

The following quantities $\vec{r}(t = 0), \vec{v}(t = 0)$ are known. As there's conservation of angular momentum, 

$$r^2 \dot \theta = \ell_0$$

Where $\ell_0$ is the specific angular momentum ($\ell_0 = L_0 / m)$, one can reckon $\ell_0$ and deduce eccentricity of the orbit path,

$$ e = \sqrt{1 + \frac{2 \epsilon \ell_0^2}{\mu^2}} $$ 

Where $\epsilon = \frac{v^2}{2} - \frac{\mu}{r}$ is the specific total energy  (Vis-viva relation) and $\mu = (m+M)G \approx MG$ is the gravitational parameter. Finally, compute the semi latus rectum with 

$$ q = \frac{\ell_0^2}{\mu} $$

We will test out different input parameters and study according trajectories.

## Post-newtonian expansion is only valid for weak gravitational field

According to theory, relativistic corrections brought by post-newtonian expansion is only valid for a weak gravitational field. Let's recall the classical gravitational field

$$ \Phi = - G \frac{M}{r^2} $$ 

To weaken $\Phi$, we can only adopt two strategies : decreasing $M$ or increasing $r$. As we are dealing with a black hole, $M$ is condemned. The only solution is to search values for $r$ that are sufficiently interesting. Let's be a bit descriptive : 

* $r \leq R_S$ : at this distance, the gravitational field is very strong. Post-newtonian approximation isn't valuable here.
* $r \in (R_S, \frac{3}{2} R_S]$ : at $1.5R_S$ is the **photon sphere**. Orbits of massive objects should be unstable.
* $r \in (\frac{3}{2} R_S, 3R_S)$ : $3R_S$ is the first stable orbit. Approximation done using $r_-$.
* $r \in (3R_S,+\infty)$ : approximation done using $r_+$.

**Notes** 

Attention : 

<a href="https://www.phys.ufl.edu/courses/phz6607/fall20/Reports/Jinye_Yang_Post-Newtonian_Theory.pdf">L'approximation post-newtonienne n'est vrai que pour un weak gravitational field</a>

À faire :

- RM,RP : 
    - Voir comment évolue l'excentricité pour la trouver égale à 0.
    - Voir comment évolue le SLR pour voir si on ne passe pas en-dessous de $R_S$.

Pour continuer, 

- Tirer un missile change la vitesse de mon vaisseau (conservation de la qté de mouvement)  $\rightarrow$ ça change mon moment angulaire $\rightarrow$ change $(q,e)$ 
- De même si je reçois un missile.

In [1]:
from pygame import Vector2
import numpy as np
import matplotlib.pyplot as plt

pygame 2.6.0 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# physical constants
G = 6.67*10E-11
c = 1
R_S = 1

# scaling factors
alpha = 10E-18

# physical quantities
M = 1/(2*G)
m = alpha * M
mu = M * G

In [50]:
## utils - may not be util anymore
def Vec2Polar(r, theta):
    return Vector2.from_polar((r, theta))

class VelVec2Polar(Vector2):
    def __init__(self, rdot, thetadot, 
                 position = Vector2.from_polar((0.,0.))):
        super().__init__()
        self.r, self.theta = position.as_polar()
        self.r_dot = rdot
        self.theta_dot = thetadot
        self.update_cartesian()  # Conversion initiale polaire -> cartésienne

    def update_cartesian(self):
        """Projette les vitesses polaires en coordonnées cartésiennes."""
        # Conversion du vecteur vitesse polaire en vecteur vitesse cartésien
        vx = self.r_dot * np.cos(self.theta) - (self.r * self.theta_dot) * np.sin(self.theta)
        vy = self.r_dot * np.sin(self.theta) + (self.r * self.theta_dot) * np.cos(self.theta)
        
        # Met à jour les composantes x et y du vecteur vitesse cartésien
        self.x = vx
        self.y = vy
    
    def magnitude(self):
        """Calcul de la magnitude en coordonnées polaires."""
        return np.sqrt(self.r_dot**2 + (self.r * self.theta_dot)**2)
    
    def update_polar(self, r, r_dot, theta, theta_dot):
        """Met à jour r, r_dot, theta et theta_dot, puis recalcule les coordonnées cartésiennes."""
        self.r = r
        self.r_dot = r_dot
        self.theta = theta
        self.theta_dot = theta_dot
        self.update_cartesian()

In [165]:
# input parameters
r0 = Vec2Polar(r = 10 * R_S, theta = 0.)
v0 = VelVec2Polar(rdot = 0. * c, thetadot = 0.001*c, position = r0)

In [166]:
# system utils
l0_polar = lambda r,v : r.magnitude()**2 * v.theta_dot
l0 = l0_polar(r0, v0) # conservation of angular momentum 

def specific_energy_polar(r,v, mu = mu):
    return v.magnitude()**2 / r.magnitude()    -   mu / r.magnitude()
    
def eccentricity_polar(r,v, l0 = l0, mu = mu): 
    k = 2 * specific_energy_polar(r,v,mu) * l0 ** 2
    return np.sqrt(1 + k/mu**2 )

def semi_latus_rectum_polar(mu=mu, l0=l0):
    return (l0**2) / mu

def rp(theta, e, q, R_S = R_S):
    # ( a + c) / b
    a = q / 2
    b = 1 + e * np.cos(theta)
    
    c1 = (q**2)/4
    c2 = (3/2 * q * R_S * (1 + e * np.cos(theta)))
    c = np.sqrt( c1 - c2 ) 
    
    return (a + c) / b

def rm(theta, e, q, R_S = R_S):
    # ( a - c) / b
    a = q / 2
    b = 1 + e * np.cos(theta)

    c1 = (q**2)/4
    c2 = (3/2 * q * R_S * (1 + e * np.cos(theta)))
    c = np.sqrt( c1 - c2 ) 
    
    return (a - c) / b

Given the input parameters, we do compute (q,e).

In [167]:
e = eccentricity_polar(r0, v0)
q = semi_latus_rectum_polar()

print(f"{l0 = },\n{e = },\n{q = } ")

l0 = 0.1,
e = np.float64(0.9979983967922995),
q = 0.020000000000000004 


And finally, using leapfrog, compute $r$ and $\theta$